In [34]:
import numpy as np
names = ['A', 'B', 'C', 'D', 'E']
qv = np.random.random((5, 5))
ss = np.random.random((5, 5))
np.savetxt('q_values.csv', qv, fmt='%.3f', delimiter='\t', header='\t'.join(names), comments='')
np.savetxt('sim_scores.csv', ss, fmt='%.3f', delimiter='\t', header='\t'.join(names), comments='')

In [64]:
import requests, os, json
CyREST = "http://localhost:1234/"
# write.csv(matrix, sep=”,”, row.names=True, column.names=True)
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}
paths = [
    os.path.join(os.getcwd(), "q_values.csv"),
     os.path.join(os.getcwd(), "sim_scores.csv")
]

In [65]:
data = {
  'files': paths,
  'delimiter': 'TAB',
  'undirected': True,
  'ignoreZeros': True,
  'interactionName': 'interacts with',
  'rowNames': False,
  'columnNames': True,
  'removeColumnPrefix': False
}
print(data)

{'files': ['/Users/bsettle/git/aMatReader/samples/q_values.csv', '/Users/bsettle/git/aMatReader/samples/sim_scores.csv'], 'delimiter': 'TAB', 'undirected': True, 'ignoreZeros': True, 'interactionName': 'interacts with', 'rowNames': False, 'columnNames': True, 'removeColumnPrefix': False}


In [66]:
# Execute post, import matrix files to a single network
res = requests.post("http://localhost:1234/aMatReader/v1/import", data = json.dumps(data), headers=HEADERS)
res = res.json()
suid = res['data']['suid']

# perform filtering via Cytoscape Automation
edges = requests.get('http://localhost:1234/v1/networks/{SUID}/edges'.format(SUID=suid))
edges = edges.json()

# loop through edge SUIDs and remove it if the q_val is below a certain threshold
# To get edge attributes, use:
for edge in edges:
    attrs = requests.get('http://localhost:1234/v1/networks/{networkSUID}/tables/defaultedge/rows/{edgeSUID}'
                     .format(networkSUID=suid, edgeSUID=edge), headers=HEADERS)
    attrs = attrs.json()
    if attrs['q_values.csv'] < .5:
        requests.delete('http://localhost:1234/v1/networks/{SUID}/edges/{edgeSUID}'.
                        format(SUID=suid, edgeSUID=edge), headers=HEADERS)
